### CM1 organization gradient standard output

In [ ]:
import sys
import os
import numpy as np
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import metpy.calc as mpc
from metpy.units import units
# for animations
from PIL import Image

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# aggregation indices
os.chdir('/data2/willytsai/python_module')
import conorgidx_revise as agi
from SCAI_calc4obj import labeled_obj, SCAI_calc
from COP_calc4obj import COP
from MICA_calc4obj import MICA

In [ ]:
def running_mean(y,window_N):
    y_avg = np.zeros(len(y))
    avg_mask = np.ones(window_N) / window_N

    y_avg = np.convolve(y, avg_mask, 'same')
    y_avg[-window_N:] = y[-window_N:]
    y_avg[:window_N] = y[:window_N]
    return y_avg    

In [ ]:
def read_data(data_dir,t_start=0,t_end=721):
    os.chdir(data_dir)
    print(data_dir)
    file_name = glob('*nc')
    data_zon = xr.open_dataset('cm1out_zonmean.nc')
    data_3d = xr.open_dataset('cm1out_sub3d.nc')
    data_2d = xr.open_dataset('cm1out_2Dvars.nc')
    
    #3d data
#    th3d = data_3d.th[t_start:t_end,:45,:,:]
    v3d = data_3d.vinterp[t_start:t_end,:,:,:]
    u3d = data_3d.uinterp[t_start:t_end,:,:,:]
    w3d = data_3d.winterp[t_start:t_end,:,:,:]
    
    # zonal mean data
    th = data_zon.th[t_start:t_end,:,:,:]
    qv = data_zon.qv[t_start:t_end,:,:,:]
#     u = data_zon.uinterp[t_start:t_end,:,:,:]
#     v = data_zon.vinterp[t_start:t_end,:,:,:]
#     w = data_zon.winterp[t_start:t_end,:,:,:]
    qc = data_zon.qc[t_start:t_end,:,:,:]
    qi = data_zon.qi[t_start:t_end,:,:,:]
    prs = data_zon.prs[t_start:t_end,:,:,:]
    # 2d data
    prate = data_2d.prate[t_start:t_end,:,:]
    pwat = data_2d.pwat[t_start:t_end,:,:]
    cwp = data_2d.cwp[t_start:t_end,:,:]
    cape = data_2d.cape[t_start:t_end,:,:]
    cin = data_2d.cin[t_start:t_end,:,:]
    
    # temperature conversion
    T = th/((1000*100/prs)**(287.15/1004))-273.15 # [degC]
    # density 
    ro = prs/(287.15*(T+273.15))

    x_dim = data_2d.lon
    y_dim = data_2d.lat
    #div = np.gradient(v,y_dim*1000,axis=2) # du/dx+dv/dy 
    #vor = -np.gradient(u,y_dim*1000,axis=2) # -du/dy+dv/dx
    #relative humidty calculation, Buck (1996)
    es_liq = 0.61121*np.exp((18.678-T/234.5)*(T/(257.14+T)))*10 #[hpa]
    es_ice = 0.61115*np.exp((23.036-T/333.7)*(T/(279.82+T)))*10

    qs_liq = 0.622*es_liq/(prs/100-es_liq)
    qs_ice = 0.622*es_ice/(prs/100-es_ice)

    rh = qv/qs_liq
    rh_ice = qv/qs_ice

    rh = rh.values.flatten()
    rh_ice = rh_ice.values.flatten()
    T_test = T.values.flatten()

    rh[T_test<0] = rh_ice[T_test<0]
    rh = rh.reshape((T.shape[0],T.shape[1],T.shape[2],T.shape[3]))
    rh = xr.DataArray(rh,coords=[T.time,T.lev,T.lat,T.lon],dims=['time','lev','lat','lon'])
    
#    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u,v,w,div,vor,cape,cin,prs,ro,th3d,v3d,u3d)
    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u3d,v3d,w3d,cape,cin,prs,ro)

In [ ]:
# get weather or weather2 
server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/'
exp_name = 'CM1_RAD4K_RCEP_240x240_S1_U104km'
exp_short = 'LS_U05'
#platform = 'weather'
os.chdir(server + exp_name)
os.system('mkdir -p fig') # create a figure archive
fig_dir = server+'/'+exp_name+'/fig/' 

In [ ]:
(th,T,qv,qc,qi,rh,prate,pwat,cwp,u3d,v3d,w3d,cape,cin,prs,ro) = read_data(server+exp_name
                                                        ,t_start=360,t_end=721)

In [ ]:
# #adding SFX for SFXint case
# data = xr.open_dataset('cm1out_SFX.nc')
# qfx = data.qfx*2.5e6
# hfx = data.hfx

In [ ]:
x_dim = pwat.lon
y_dim = pwat.lat
z_dim = T.lev
t_dim = np.arange(len(T.time))/3

In [ ]:
# check theta profile
fig = plt.figure(figsize=(5,5))
plt.plot(th[0,:,:,0].mean(axis=1),z_dim,'k',label='INIT')
plt.plot(th[-1,:,:,0].mean(axis=1),z_dim,'b',label='EQUI')
plt.legend()
plt.ylim([0,22]);plt.xlim([290,450])
plt.xlabel('Theta [K]',fontsize=13)
plt.ylabel('Height [km]',fontsize=13)
fig.savefig(fig_dir+'theta_profile.pdf',bbox_inches='tight',transparent=False)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
ax[0].plot(th[-72:,:,:,0].mean(axis=(0,2))-th[0,:,:,0].mean(axis=1),z_dim,'k')
ax[1].plot(1000*(qv[-72:,:,:,0].mean(axis=(0,2))-qv[0,:,:,0].mean(axis=1)),z_dim,'k')
ax[2].plot((rh[-72:,:,:,0].mean(axis=(0,2))-rh[0,:,:,0].mean(axis=1))*100,z_dim,'k')

ax[0].set_ylim([0,25]);#plt.xlim([-11,7])
ax[0].set_xlabel('Theta [K]',fontsize=13)
ax[0].set_ylabel('Height [km]',fontsize=13)
      
ax[1].set_ylim([0,25]);#plt.xlim([-11,7])
ax[1].set_xlabel('qv [g/kg]',fontsize=13)
ax[1].set_ylabel('Height [km]',fontsize=13)

ax[2].set_ylim([0,25]);#plt.xlim([-11,7])
ax[2].set_xlabel('RH [%]',fontsize=13)
ax[2].set_ylabel('Height [km]',fontsize=13)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
ax.plot(t_dim,pwat.mean(['lon','lat']),'-ob',zorder=1,label='PREC') # P in W/m^2
#ax.plot(t_dim,np.ones(len(t_dim))*385,'k',label='LHF') # TFX W/m^2
ax.legend()
ax.set_xlabel('Time [hr]',fontsize=13)
#ax.set_ylabel('Flux [W/m$^2$]',fontsize=13)
ax.set_title(exp_name)
# mm/d axis
#ax2 = ax.twinx()
#ax2.plot(t_dim,prate.mean(['lon','lat']),color='b',linewidth=.5,zorder=-1) # P in W/m^2
#ax2.set_ylabel('[mm/d]',fontsize=13)
fig.savefig(fig_dir+'PWAT_timeseries.pdf',fontsize=13)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
ax.plot(t_dim,prate.mean(['lon','lat'])*2.5e6/86400,'b',zorder=1,label='PREC') # P in W/m^2
ax.plot(t_dim,np.ones(len(t_dim))*385,'k',label='LHF') # TFX W/m^2
ax.legend()
ax.set_xlabel('Time [hr]',fontsize=13)
ax.set_ylabel('Flux [W/m$^2$]',fontsize=13)
ax.set_title(exp_name)
# mm/d axis
ax2 = ax.twinx()
ax2.plot(t_dim,prate.mean(['lon','lat']),color='b',linewidth=.5,zorder=-1) # P in W/m^2
ax2.set_ylabel('[mm/d]',fontsize=13)
fig.savefig(fig_dir+'PRATE_SFX_timeseries.pdf',fontsize=13)

In [ ]:
# # animations for U component evolution in the y-z plane

# for t in range(0,len(t_dim),3):
#     fig,ax = plt.subplots(1,1,figsize=(10,4))
#     cf = ax.contourf(y_dim,z_dim[:60],pt1[t,:60,:,0],levels=np.linspace(1,10,16),cmap='jet')
#     cbar = plt.colorbar(cf,ax=ax,shrink=0.8)
#     cbar.set_label('U [m/s]',fontsize=13)
#     cf2 = ax.contour(y_dim,z_dim[:60],pt1[t,:60,:,0],levels=[-5,-4,-3,-2,-1,1,2,3,4,5]
#                      ,colors=['k'],linewidths=0.6)
#     ax.set_xlabel('y [km]',fontsize=13)
#     ax.set_ylabel('Height [km]',fontsize=13)
    
#     ax.set_title(exp_name+',  Time= '+ str(np.round(t/3,2)) + ' (h)',fontsize=14)

# #     # save figs
#     if (t < 10):
#         plt.savefig(fig_dir+'/'+exp_name+'_000'+str(t)+'.png',dpi=300)
#     elif (10 <= t) and (t < 100):
#         plt.savefig(fig_dir+'/'+exp_name+'_00'+str(t)+'.png',dpi=300)
#     elif (100 <= t) and (t < 1000):
#         plt.savefig(fig_dir+'/'+exp_name+'_0'+str(t)+'.png',dpi=300)
#     else:
#         plt.savefig(fig_dir+'/'+exp_name+'_'+str(t)+'.png',dpi=300)
#     plt.close(fig)
# # end of loop
# # save figures as gif
# fp_in = fig_dir+'/*.png'
# fp_out = fig_dir+'/'+exp_name+'.gif'

In [ ]:
# img, *imgs = [Image.open(f) for f in sorted(glob(fp_in))]
# img.save(fp=fp_out, format='GIF', append_images=imgs,
#           save_all=True, duration=200, loop=0)

In [ ]:
# soundings over 6 places
y_pos = [13,39,52,65,91]
color_snds = ['green','purple','purple','purple','green']
ncol = [0,1,2,3,0,1,2,3]
nrow = [0,0,0,0,1,1,1,1]
pos_label = ['NS','A','A','A','NS']

fig,ax = plt.subplots(1,1,figsize=(6,6))

for n,(T,qv,prs) in enumerate(zip([T],[qv],
                                 [prs])):
    
    mse = 1004*(T[-360:,:,:,0]+273.15).mean(axis=0)+2.5e6*qv[-360:,:,:,0].mean(axis=0) \
            +9.8*np.tile(z_dim*1000,reps=(len(y_dim),1)).swapaxes(0,1)
    dse = 1004*(T[-360:,:,:,0]+273.15).mean(axis=0) \
            +9.8*np.tile(z_dim*1000,reps=(len(y_dim),1)).swapaxes(0,1)
        
    for p,y in enumerate(y_pos):
    
        # smse
        qvs = mpc.saturation_mixing_ratio(prs[-360:,:,y,0].mean(axis=(0)).values/100*units('mbar')
                            ,T[-360:,:,y,0].mean(axis=(0)).values*units('degC'))
        smse = 1004*(T[-360:,:,y,0]+273.15).mean(axis=0)+2.5e6*qvs \
                +9.8*z_dim*1000
        
        ax.plot(dse[:,y]/1000,z_dim,color=color_snds[p],linestyle='--')
        ax.plot(mse[:,y]/1000,z_dim,color=color_snds[p],linestyle='-',label=pos_label[p])
        ax.plot(smse/1000,z_dim,color=color_snds[p],linestyle='-.')

        ax.set_title(exp_name)
        ax.set_ylim([0,12])
        ax.set_xlim([325,350])
        ax.grid(linestyle=':')
ax.set_xlabel('MSE')
ax.legend()

In [ ]:
# # hovemuller diagram
# fig,ax = plt.subplots(3,1,figsize=(13,9))
# cf = ax[0].pcolor(t_dim,y_dim,pwat.mean('lon').T,cmap='Spectral',vmin=50,vmax=60)
# cbaxes = inset_axes(ax[0], width="2%",height="70%",loc='lower left',
#            bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
#            borderpad=0)
# cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical')
# cbar.set_label('PW [mm]')

# ax[0].contour(t_dim,y_dim,prate.mean('lon').T,levels=[20],colors=['k',]
#              ,linewidths=1)
# ax[1].plot(t_dim,prate.mean(('lon','lat')))
# ax[1].set_xlim([0,t_dim[-1]])#;ax[1].set_ylim([7,18])
# ax[2].plot(t_dim,pwat.mean(('lon','lat')))
# ax[2].set_xlim([0,t_dim[-1]])#;ax[2].set_ylim([42,46])

# ax[0].set_ylabel('[km]',fontsize=12)
# ax[1].set_ylabel('[mm/d]',fontsize=12)
# ax[2].set_ylabel('[mm]',fontsize=12)
# ax[0].set_title(exp_name,fontsize=14)
# ax[0].text(10,0.8*y_dim[-1],'contour=20[mm/d]',color='r',fontsize=12)
# ax[1].grid(linestyle=':',linewidth=1)
# ax[2].grid(linestyle=':',linewidth=1)
# #fig.savefig(fig_dir+'PW_PREC_hovmuller.pdf',bbox_inches='tight')
# #plt.close()
# print('PW_PREC_hovmuller.pdf')

In [ ]:
# fig,ax = plt.subplots(1,1,figsize=(13,3))
# ax.plot(t_dim,(qfx+hfx).mean(('lon','lat'))[:2880],label='qfx+hfx')
# ax.plot(t_dim,(qfx).mean(('lon','lat'))[:2880],'k',label='qfx')
# ax.plot(t_dim,(hfx).mean(('lon','lat'))[:2880],'--k',label='hfx')

# ax.set_ylabel('SFX [W/m$^2$]',fontsize=12)
# ax.legend()
# #fig.savefig(fig_dir+'SFX_hovmuller.pdf',bbox_inches='tight')
# print('SFX_hovmuller.pdf')

In [ ]:
# # hovemuller diagram
# fig,ax = plt.subplots(3,1,figsize=(10,10))
# cf = ax[0].pcolor(t_dim,y_dim,u[:,1,:,:].mean('lon').T,cmap='Spectral')
# cbaxes = inset_axes(ax[0], width="2%",height="70%",loc='lower left',
#            bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
#            borderpad=0)
# cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical')
# cbar.set_label('U [m/s]')

# cf = ax[1].pcolor(t_dim,y_dim,v[:,1,:,:].mean('lon').T,cmap='Spectral')
# cbaxes = inset_axes(ax[1], width="2%",height="70%",loc='lower left',
#            bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[1].transAxes,
#            borderpad=0)
# cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical')
# cbar.set_label('V [m/s]')
# ax[1].contourf(t_dim,y_dim,prate.mean('lon').T/24,levels=np.linspace(1,5,5),colors=['k'])

# llconv = np.trapz(np.gradient(v[:,:8,:,:].mean('lon'),y_dim*1000,axis=2),z_dim[:8],axis=1)/z_dim[8].values # low-level mean convergence (sfc to 1km)
# cf = ax[2].pcolor(t_dim,y_dim,llconv.T,cmap='Spectral')
# cbaxes = inset_axes(ax[1], width="2%",height="70%",loc='lower left',
#            bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[2].transAxes,
#            borderpad=0)
# cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical')
# cbar.set_label('low-level divergence [1/s]')

# ax[0].set_ylabel('[km]',fontsize=13)
# ax[1].set_ylabel('[km]',fontsize=13)
# ax[2].set_ylabel('[km]',fontsize=13)
# ax[2].set_xlabel('[hr]',fontsize=13)
# ax[0].set_title(exp_name,fontsize=12)

# # ax[1].grid(linestyle=':',linewidth=1)
# # ax[2].grid(linestyle=':',linewidth=1)
# #fig.savefig(fig_dir+'PW_PREC_LLCOV_hovmuller.pdf',bbox_inches='tight')
# #plt.close()
# print('PW_PREC_LLCOV_hovmuller.pdf')

In [ ]:
# # vertical profile t-z cross-section: T, qv, qi+qv, rh
# fig,ax = plt.subplots(4,1,figsize=(13,12))
# cf0=ax[0].contourf(t_dim,z_dim,T.mean(('lon','lat')).T)
# cbar0=plt.colorbar(cf0,ax=ax[0]);cbar0.set_label('theta [K]')
# cf1=ax[1].contourf(t_dim,z_dim,1000*qv.mean(('lon','lat')).T)
# cbar1=plt.colorbar(cf1,ax=ax[1]);cbar1.set_label('qv [g/kg]')
# cf2=ax[2].contourf(t_dim,z_dim,np.log10(1000*(qc+qi).mean(('lon','lat')).T))
# cbar2=plt.colorbar(cf2,ax=ax[2]);cbar2.set_label('qc+qi')
# cf3=ax[3].contourf(t_dim,z_dim,100*rh.mean(('lon','lat')).T)
# cbar3=plt.colorbar(cf3,ax=ax[3]);cbar3.set_label('RH [%]')
# fig.savefig(fig_dir+'Thermo_crosssect_tz.pdf',bbox_inches='tight')
# plt.close()
# print('Thermo_crosssect_tz.pdf')

### Snapshot of evolutions

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(8,4))

t=306

cf = ax[0].pcolor(x_dim,y_dim,prate[t,:,:]/24,vmin=.5,cmap='jet');cf.set_clim(.5,60)
cf = ax[1].pcolor(x_dim,y_dim,prate[t+6,:,:]/24,vmin=.5,cmap='jet');cf.set_clim(.5,60)
cf = ax[2].pcolor(x_dim,y_dim,prate[t+12,:,:]/24,vmin=.5,cmap='jet');cf.set_clim(.5,60)
cf = ax[3].pcolor(x_dim,y_dim,prate[t+18,:,:]/24,vmin=.5,cmap='jet');cf.set_clim(.5,60)
cf.cmap.set_under('white')

ax[0].set_title('t= '+str(t//3)+' (h)',loc='right',fontsize=12)
ax[1].set_title('t= '+str((t+6)//3)+' (h)',loc='right',fontsize=12)
ax[2].set_title('t= '+str((t+12)//3)+' (h)',loc='right',fontsize=12)
ax[3].set_title('t= '+str((t+18)//3)+' (h)',loc='right',fontsize=12)
ax[0].text(30,450,exp_short,color='r',fontweight='bold')

ax[0].set_xlabel('[km]',fontsize=13)
ax[1].set_xlabel('[km]',fontsize=13)
ax[2].set_xlabel('[km]',fontsize=13)
ax[3].set_xlabel('[km]',fontsize=13)
ax[0].set_ylabel('[km]',fontsize=13)

ax[0].set_aspect('equal')
ax[1].set_aspect('equal')
ax[2].set_aspect('equal')
ax[3].set_aspect('equal')
plt.tight_layout(h_pad=0.2)

fig.savefig(fig_dir+'prate_snapshots.pdf',bbox_inches='tight')
#plt.close()
print('prate_snapshots.pdf')

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(16,4))

t=306

cf = ax[0].pcolor(pwat[t,:,:],vmin=35,cmap='jet');cf.set_clim(50,60)
cf = ax[1].pcolor(pwat[t+6,:,:],vmin=35,cmap='jet');cf.set_clim(50,60)
cf = ax[2].pcolor(pwat[t+12,:,:],vmin=35,cmap='jet');cf.set_clim(50,60)
cf = ax[3].pcolor(pwat[t+18,:,:],vmin=35,cmap='jet');cf.set_clim(50,60)
cf.cmap.set_under('white')

ax[0].set_title('t= '+str(t//3)+' (h)',loc='right',fontsize=12)
ax[1].set_title('t= '+str((t+6)//3)+' (h)',loc='right',fontsize=12)
ax[2].set_title('t= '+str((t+12)//3)+' (h)',loc='right',fontsize=12)
ax[3].set_title('t= '+str((t+18)//3)+' (h)',loc='right',fontsize=12)
ax[0].set_xlabel('[km]',fontsize=13)
ax[0].set_ylabel('[km]',fontsize=13)
plt.tight_layout()

#fig.savefig(fig_dir+'pwat_snapshots.pdf',bbox_inches='tight')
#plt.close()
print('pwta_snapshots.pdf')

In [ ]:
#Z = np.tile(z_dim*1000,reps=(len(t_dim),len(y_dim),1)).swapaxes(1,2)

In [ ]:
# # MSE time-series
# vint_mse = np.zeros((len(t_dim),216))
# vint_Lqv = np.copy(vint_mse)
# vint_CpT = np.copy(vint_mse)

# for n,(T,qv) in enumerate(zip([T],[qv])):
#     vint_mse[:,:] = np.trapz(1004*(T.squeeze()+273.15)+9.8*Z*1000+2.5e6*qv.squeeze(),z_dim*1000,axis=1).squeeze()
#     vint_Lqv[:,:] = np.trapz(2.5e6*qv,z_dim*1000,axis=1).squeeze()
#     vint_CpT[:,:] = np.trapz(1004*(T+273.15),z_dim*1000,axis=1).squeeze()

In [ ]:
# # decompose into <T> & <qv> contributions
# fig = plt.figure(figsize=(10,5))

# plt.plot(t_dim,vint_CpT[:,:].mean(axis=1)/1000-vint_CpT[0,:].mean()/1000,color='r',label=exp_name,
#             linewidth=1.8)
# plt.legend(frameon=False)
    
# plt.xlim([0,240])
# plt.ylabel('Vint_CpT [KJ/kg*m]')
# plt.xlabel('Time [hr]')
# plt.grid(linestyle=':')

In [ ]:
# # decompose into <T> & <qv> contributions
# fig = plt.figure(figsize=(10,5))

# plt.plot(t_dim,vint_Lqv[:,:].mean(axis=1)/1000-vint_Lqv[0,:].mean()/1000,color='r',label=exp_name,
#             linewidth=1.8)
# plt.legend(frameon=False)
    
# plt.xlim([0,240])
# plt.ylabel('Vint_Lqv [KJ/kg*m]')
# plt.xlabel('Time [hr]')
# plt.grid(linestyle=':')

In [ ]:
data_CTRL = xr.open_dataset('../CM1_RAD4K_MPI_SFXint/cm1out_2Dvars.nc')
prate_CTRL = 24*3600*data_CTRL.prate[-72:,:,:].mean().values # mm/d

In [ ]:
# plotting zonal mean
# y cross-section
marker_label=['s','o']

fig,ax = plt.subplots(2,1,figsize=(9,6))

pwat_mn = np.nanmean(pwat[-360:,:,:],axis=(0,2))
pwat_p1std = np.nanmean(pwat[-360:,:,:],axis=(0,2)) + np.nanstd(pwat[-360:,:,:],axis=(0,2))/np.sqrt(360)
pwat_m1std = np.nanmean(pwat[-360:,:,:],axis=(0,2)) - np.nanstd(pwat[-360:,:,:],axis=(0,2))/np.sqrt(360)
ax[0].plot(y_dim,pwat_mn,color='r'
          ,label=exp_name,linewidth=2)
ax[0].fill_between(y_dim,pwat_m1std,pwat_mn,facecolor='gainsboro')
ax[0].fill_between(y_dim,pwat_p1std,pwat_mn,facecolor='gainsboro')

prate_mn = np.nanmean(prate[-360:,:,:],axis=(0,2))
prate_p1std = np.nanmean(prate[-360:,:,:],axis=(0,2)) + np.nanstd(prate[-360:,:,:],axis=(0,2))/np.sqrt(360)
prate_m1std = np.nanmean(prate[-360:,:,:],axis=(0,2)) - np.nanstd(prate[-360:,:,:],axis=(0,2))/np.sqrt(360)

ax[1].plot(y_dim,prate_mn,color='b'
          ,label=exp_name,linewidth=2)
ax[1].fill_between(y_dim,prate_m1std,prate_mn,facecolor='gainsboro')
ax[1].fill_between(y_dim,prate_p1std,prate_mn,facecolor='gainsboro')
# add CTRL prate
prate_CTRL = np.ones(len(y_dim))*prate_CTRL
ax[1].plot(y_dim,prate_CTRL,color='k')

ax[0].legend(frameon=False,fontsize=8,ncol=2);
ax[0].set_xlabel('y direction [km]',fontsize=14)#;ax[0].set_ylim([57,63])
ax[0].set_ylabel('CWV [mm]',fontsize=14)
ax[1].set_xlabel('y direction [km]',fontsize=14)#;ax[1].set_ylim([5,15])
ax[1].set_ylabel('Prec [mm/d]',fontsize=14)
ax[0].set_xlim([0,y_dim[-1]])
ax[1].set_xlim([0,y_dim[-1]])
ax[1].set_ylim([6,20])
plt.tight_layout(h_pad=1)
ax[0].grid(linestyle=':')
ax[1].grid(linestyle=':')
ax[1].legend(frameon=False,fontsize=8,ncol=2);
ax[0].tick_params(labelsize=11)
ax[1].tick_params(labelsize=11)

ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)

fig.savefig(fig_dir+'PWAT_PREC_crossection.pdf',dpi=200,bbox_inches='tight')
#plt.close()
#print('PWAT_PREC_crossection.pdf')

In [ ]:
# fig = plt.figure(figsize=(12,3))

# plt.contourf(y_dim,z_dim,u[719,:,:,0])
# plt.colorbar()
# plt.title(exp_name + ', U wind [m/s]')
# plt.ylim([0,10])
# plt.ylabel('Height [km]',fontsize=13)
# plt.xlabel('y direction [km]',fontsize=13)

In [ ]:
# fig = plt.figure(figsize=(12,3))

# plt.contourf(y_dim,z_dim,v[-100:,:,:,0].mean(axis=0))
# plt.colorbar()
# plt.title(exp_name + ', V wind [m/s]')
# plt.ylim([0,10])
# plt.ylabel('Height [km]',fontsize=13)
# plt.xlabel('y direction [km]',fontsize=13)

In [ ]:
# streamfunction, anelastic + boussinesq==> grad(ro*V) = 0
strmf = np.zeros((360,len(z_dim),len(y_dim)))
v = v3d.mean('lon')

for t in range(360):
    for n in range(len(z_dim)):
        #v_mean = v[t,:n,:].mean(axis=1)
        strmf[t,n,:] = np.trapz(ro[t,:n,:,0]*(v[t,:n,:]),z_dim[:n]*1000,axis=0) 

In [ ]:
# # U cross section
# fig,ax = plt.subplots(len(exp_name)-1,1,figsize=(8,10))

# for n,u in enumerate([u1]):
#     cf=ax[n].contourf(y_dim,z_dim,u[:,:,0],levels=np.linspace(1,15,8),linewidths=2,cmap='RdBu')
#     #ax[n].contour(y_dim,z_dim,v[-360:,:,:,0].mean(axis=0),levels=[0],colors='k')
#     #ax[n].contour(y_dim,z_dim,T[-360:,:,:,0].mean(axis=0),levels=['40'],linewidths=2,colors=['grey'])
#     #ax[n].set_ylim([0,18])
#     if n == 4:
#         cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
#                    bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[n].transAxes,
#                    borderpad=0) 
#         cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical',shrink=1)
#         cbar.set_label('U [m/s]')

#     ax[n].set_ylabel('height [km]',fontsize=13)
#     ax[n].grid(linestyle=':')
#     ax[n].set_title(exp_label[n+1],fontsize=14,loc='right')
#     ax[n].set_ylim([0,6])    
#     ax[n].set_xlabel('y direction [km]',fontsize=13)
# plt.tight_layout()
# fig.savefig(fig_out+'Ushear_crossection.png',dip=200,bbox_inches='tight')
# plt.close()
# print('Ushear_crossection.png')

In [ ]:
# # V cross section
# fig,ax = plt.subplots(len(exp_name)-1,1,figsize=(8,10))

# for n,v in enumerate([v2,v3,v4,v5,v6]):
#     cf=ax[n].contourf(y_dim,z_dim,v[-360:,:,:,0].mean(axis=0),levels=np.linspace(-1.5,1.5,21),linewidths=2,cmap='RdBu')
#     ax[n].contour(y_dim,z_dim,v[-360:,:,:,0].mean(axis=0),levels=[0],colors='k')
#     #ax[n].contour(y_dim,z_dim,T[-360:,:,:,0].mean(axis=0),levels=['40'],linewidths=2,colors=['grey'])
#     ax[n].set_ylim([0,18])
#     if n == 0:
#         cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
#                    bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
#                    borderpad=0) 
#         cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical',shrink=1)

#     ax[n].set_ylabel('height [km]',fontsize=13)
#     ax[n].grid(linestyle=':')
#     ax[n].set_title(exp_name[n+1],fontsize=14)
    
# ax[n].set_xlabel('y direction [km]',fontsize=13)
# plt.tight_layout()

In [ ]:
# # soundings over 6 places
# y_pos = [26,52,78,104,130,156,182]
# color_snds = ['green','r','purple','purple','purple','r','green']
# ncol = [0,1,2,3,0,1,2,3]
# nrow = [0,0,0,0,1,1,1,1]
# pos_label = ['NS','EG1','S','S','S','EG2','NS']

# fig,ax = plt.subplots(2,4,figsize=(20,10))

# for n,(T,qv,prs) in enumerate(zip([T1,T2,T3,T4,T5,T6],[qv1,qv2,qv3,qv4,qv5,qv6],
#                                  [prs1,prs2,prs3,prs4,prs5,prs6])):
    
#     mse = 1004*(T[-360:,:,:,0]+273.15).mean(axis=0)+2.5e6*qv[-360:,:,:,0].mean(axis=0) \
#             +9.8*np.tile(z_dim*1000,reps=(len(y_dim),1)).swapaxes(0,1)
#     dse = 1004*(T[-360:,:,:,0]+273.15).mean(axis=0) \
#             +9.8*np.tile(z_dim*1000,reps=(len(y_dim),1)).swapaxes(0,1)
        
#     for p,y in enumerate(y_pos):
    
#         # smse
#         qvs = mpc.saturation_mixing_ratio(prs[-360:,:,y,0].mean(axis=(0)).values/100*units('mbar')
#                             ,T[-360:,:,y,0].mean(axis=(0)).values*units('degC'))
#         smse = 1004*(T[-360:,:,y,0]+273.15).mean(axis=0)+2.5e6*qvs \
#                 +9.8*z_dim*1000
        
#         ax[nrow[n],ncol[n]].plot(dse[:,y]/1000,z_dim,color=color_snds[p],linestyle='--')
#         ax[nrow[n],ncol[n]].plot(mse[:,y]/1000,z_dim,color=color_snds[p],linestyle='-',label=pos_label[p])
#         ax[nrow[n],ncol[n]].plot(smse/1000,z_dim,color=color_snds[p],linestyle='-.')

#         ax[nrow[n],ncol[n]].set_title(exp_label[n])
#         ax[nrow[n],ncol[n]].set_ylim([0,18])
#         ax[nrow[n],ncol[n]].set_xlim([300,360])
#         ax[nrow[n],ncol[n]].grid(linestyle=':')
# ax[0,0].legend()

In [ ]:
# # soundings over 6 places, cape and cin values
# y_pos = [26,52,78,104,130,156,182]
# color_snds = ['green','r','purple','purple','purple','r','green']
# #nrow = [0,0,1,1,2,2,3]
# #ncol = [0,1,0,1,0,1,0]
# pos_label = ['NS','EG1','S','S','S','EG2','NS']

# fig,ax = plt.subplots(1,2,figsize=(10,5))

# for n,(cape,cin) in enumerate(zip([cape1,cape2,cape3,cape4,cape5,cape6]
#                                   ,[cin1,cin2,cin3,cin4,cin5,cin6])):    
#     for p,y in enumerate(y_pos):
#         ax[0].plot(p,cape[-360:,y,:].mean(),'^',color=color_label[n])
#         ax[1].plot(p,cin[-360:,y,:].mean(),'o',color=color_label[n])

# ax[0].set_xticks(np.arange(len(y_pos)))
# ax[1].set_xticks(np.arange(len(y_pos)))
# ax[0].set_xticklabels(pos_label)
# ax[1].set_xticklabels(pos_label)

In [ ]:
# # Relative humdity profile
# fig,ax = plt.subplots(1,1,figsize=(5,6))
# ax.plot(rh[-360:,:,:,0].mean(axis=(0,2)),z_dim,'k')
# ax2 = ax.twiny()
# ax2.plot(1000*qv[-360:,:,:,0].mean(axis=(0,2)),z_dim,'b')
# ax2.set_xlabel('Specific humidity',fontsize=13,color='b')
# ax.set_xlabel('Relative humidity',fontsize=13)
# ax.set_ylabel('Height [m]',fontsize=13)
# ax.set_ylim([0,12]);ax.set_xlim([0.4,1])
# ax2.set_xlim([0,17])
# ax.legend([exp_name])
# #fig.savefig(fig_dir+'RH_qv_DMprofile.pdf',bbox_inches='tight',transparent=False)

In [ ]:
u3d.shape

In [ ]:
plt.contourf(y_dim,z_dim,u3d[-1,:,:,:].mean('lon'))
plt.colorbar()

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
clevs = np.linspace(-1000,1000,21)

for n,(strmf,rh) in enumerate(zip([strmf]
                         ,[rh])):
    ax.contour(y_dim,z_dim,strmf[-360:,:,:].mean(axis=0),levels=clevs,linewidths=1,colors='k')
    ax.contour(y_dim,z_dim,strmf[-360:,:,:].mean(axis=0),levels=['0'],linewidths=2,colors=['w'])
    cf = ax.pcolor(y_dim,z_dim,100*rh[-360:,:,:,0].mean(axis=0),vmin=40,vmax=100,cmap='BrBG')
    ax.set_ylim([0,18])

    ax.set_ylabel('height [km]',fontsize=13)
    ax.grid(linestyle=':')
    ax.set_title(exp_name,fontsize=14,loc='right')
    ax.set_xlabel('y direction [km]',fontsize=13)
    
    cbaxes = fig.add_axes([1.01, 0.2, 0.02, 0.6]) 
    cb = plt.colorbar(cf, cax = cbaxes)  
    cb.set_label('Relative humidity [%]',fontsize=13)
    
plt.tight_layout()
fig.savefig(fig_dir+'rh_strm_crosssection.pdf',dip=200,bbox_inches='tight')
#plt.close()
print(fig_dir+'rh_strm_crosssection.pdf')

In [ ]:
# cloud fraction 
cld_mask = np.copy(qc)*0
cld_mask[qc+qi>1e-5] = 1

fig,ax = plt.subplots(1,3,figsize=(15,5))
 
cf = ax[0].pcolor(y_dim,z_dim,qc[-360:,:,:,0].mean(axis=0)+qi[-360:,:,:,0].mean(axis=0),vmin=1e-5,vmax=7*1e-5,cmap='jet')
cbar = plt.colorbar(cf,ax=ax[0],shrink=0.7)
cbar.set_label('qc+qi')
ax[0].set_ylim([0,18])

ax[0].set_ylabel('height [km]',fontsize=13)
ax[0].grid(linestyle=':')
ax[0].set_title(exp_name,fontsize=12)

cf = ax[1].pcolor(y_dim,z_dim,cld_mask[-360:,:,:,0].mean(axis=0),cmap='jet')
cbar = plt.colorbar(cf,ax=ax[1],shrink=0.7)
cbar.set_label('cloud fraction')
ax[1].set_ylim([0,18])

cld_s = np.nanmean(cld_mask[-360:,:,90:170,0],axis=(0,2))
cld_ns = 0.5*(np.nanmean(cld_mask[-360:,:,:40,0],axis=(0,2)) + np.nanmean(cld_mask[-360:,:,-40:,0],axis=(0,2)))

ax[2].plot(cld_s,z_dim,'-k',label='S')
ax[2].plot(cld_ns,z_dim,'-r',label='NS')
ax[2].set_ylabel('height [km]',fontsize=13)
ax[2].set_ylim([0,18]);ax[2].legend()
ax[2].set_xlim([0,1])
ax[2].set_xlabel('Cloud fraction',fontsize=13)

ax[1].set_ylabel('height [km]',fontsize=13)
ax[1].grid(linestyle=':')
ax[1].set_title(exp_name,fontsize=12)
ax[0].set_xlabel('y direction [km]',fontsize=13)
ax[1].set_xlabel('y direction  [km]',fontsize=13)

plt.tight_layout()
fig.savefig(fig_dir+'qcqi_cldfrac_crosssection.pdf',dip=200,bbox_inches='tight')
#plt.close()
print('qcqi_cldfrac_crosssection.pdf')

### W variances in space and time

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
cf=plt.contourf(t_dim[-360:],y_dim,w3d[-360:,14,:,:].mean('lon').T,cmap='jet')
plt.colorbar(cf)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
cf=plt.contourf(t_dim[-360:],y_dim,w3d[-360:,14,:,:].var('lon').T,cmap='jet')
plt.colorbar(cf)